### All necessary imports

In [137]:
from lxml.html import parse
from urllib.request import urlopen
from pandas.io.parsers import TextParser

### Storing the cik and acc number in variables

In [84]:
file_obj = open("cikInformation.txt","r" )
cik = file_obj.readline().rstrip()
acc = file_obj.readline().rstrip()
accNoDash = acc.replace('-', '')

### Parsing the url 

In [96]:
url = 'https://www.sec.gov/Archives/edgar/data/'+cik+'/'+accNoDash+'/'+acc+'-index.htm'

https://www.sec.gov/Archives/edgar/data/51143/000005114313000007/0000051143-13-000007-index.htm


In [99]:
parsed = parse(urlopen(url))
doc = parsed.getroot()

In [100]:
links = doc.findall('.//a')

In [101]:
url = [lnk.get('href')for lnk in doc.findall('.//a')]

In [102]:
url[10:23]

['/Archives/edgar/data/51143/000005114313000007/ibm13q3_ex11.htm',
 '/Archives/edgar/data/51143/000005114313000007/ibm13q3_ex12.htm',
 '/Archives/edgar/data/51143/000005114313000007/ibm13q3_ex31d1.htm',
 '/Archives/edgar/data/51143/000005114313000007/ibm13q3_ex31d2.htm',
 '/Archives/edgar/data/51143/000005114313000007/ibm13q3_ex32d1.htm',
 '/Archives/edgar/data/51143/000005114313000007/ibm13q3_ex32d2.htm',
 '/Archives/edgar/data/51143/000005114313000007/0000051143-13-000007.txt',
 '/Archives/edgar/data/51143/000005114313000007/ibm-20130930.xml',
 '/Archives/edgar/data/51143/000005114313000007/ibm-20130930.xsd',
 '/Archives/edgar/data/51143/000005114313000007/ibm-20130930_cal.xml',
 '/Archives/edgar/data/51143/000005114313000007/ibm-20130930_def.xml',
 '/Archives/edgar/data/51143/000005114313000007/ibm-20130930_lab.xml',
 '/Archives/edgar/data/51143/000005114313000007/ibm-20130930_pre.xml']

In [103]:
tables = doc.findall('.//table')

In [104]:
example = tables[0]

In [105]:
rows = example.findall('.//tr')

In [106]:
#def _unpack(row, kind='td'):
#    elts = row.findall('.//%s' % kind)
#    return [val.text_content() for val in elts]

#def parse_options_data(table):
#    rows=table.findall('.//tr')
#    header = _unpack(rows[0], kind ='th')
#    data = [_unpack(r) for r in rows[1:]]
#    return TextParser(data, names = header).get_chunk()

#example_data = parse_options_data(example)
#example_data[:10]

#for r in range(len(example_data)):
#        d= example_data['Description'][r]
#        if d =='10-Q' :
#            Doc.append(example_data['Document'][r])



In [107]:
print(_unpack(rows[0], kind="th"))

['Seq', 'Description', 'Document', 'Type', 'Size']


In [108]:
Doc=[]
for t in range(len(tables)):
    example = tables[t]
    
    rows = example.findall('.//tr')
    
    def _unpack(row, kind='td'):
        elts = row.findall('.//%s' % kind)
        return [val.text_content() for val in elts]

    def parse_options_data(table):
        rows=table.findall('.//tr')
        header = _unpack(rows[0], kind ='th')
        data = [_unpack(r) for r in rows[1:]]
        return TextParser(data, names = header).get_chunk()

    example_data = parse_options_data(example)
    
    
    for r in range(len(example_data)):
        d= example_data['Description'][r]
        if d =='10-Q' :
            Doc.append(example_data['Document'][r])

In [109]:
Doc

['ibm13q3_10q.htm']